## Baum-Welch Algorithm

This algorithm is using the EM algorithm for Hidden Markov model, our goal is the get the transition probability and emission probability.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
# previous algorithm
def forward(V, a, b, initial_distribution):
    ####################################################
    # V : it is the data for visible state             #
    # a : transition probability                       #
    # b : emission probability                         #
    # initial_distribution : start that you assume     #
    ####################################################
    alpha = np.zeros((V.shape[0], a.shape[0]))        # 500 X 2
    alpha[0, :] = initial_distribution * b[:, V[0]]   # first row = initial * first visible data
 
    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
 
    return alpha

def backward(V, a, b):
    ###################################################################################
    # V : it is the data for visible state                                            #
    # a : transition probability                                                      #
    # b : emission probability                                                        #
    # backward start from assign each state 1, so we don't need initial probability.  #
    ###################################################################################
    beta = np.zeros((V.shape[0], a.shape[0]))
 
    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
 
    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
 
    return beta

first start with guessing the result of transition probability and emission probability, also the proportion of the hidden state. 

In [33]:
data = pd.read_csv('data for HMM.csv')
 
V = data['Visible'].values

Let's try to get the estimation of proportion of hidden state, transition probability and emission probability.

We can simply assume the proportion to be the same (0.5, 0.5), and 

In [62]:
# it's just a initial guess, we will get the precise transition probability and emission probability by the algorithm
a = np.ones((2, 2))
a = a / np.sum(a, axis=1)
print(a)
b = np.array(((1, 3, 5), (2, 4, 6)))
b = b / np.sum(b, axis=1).reshape((-1, 1))
print(b)
initial_distribution = np.array((0.5, 0.5))

[[0.5 0.5]
 [0.5 0.5]]
[[0.11111111 0.33333333 0.55555556]
 [0.16666667 0.33333333 0.5       ]]


In [105]:
def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0]
    T = len(V)
 
    for n in range(n_iter):
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)
 
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator
 
        gamma = np.sum(xi, axis=1)
        #print(np.sum(gamma, axis = 1))
        v = np.sum(gamma, axis = 1)/T
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
 
        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)
 
        b = np.divide(b, denominator.reshape((-1, 1)))
 
    return {"a":a, "b":b, "initial_distribution": v}

In [119]:
a = np.ones((2, 2))
a = a / np.sum(a, axis=1)

b = np.array(((1, 3, 5), (2, 4, 6)))
b = b / np.sum(b, axis=1).reshape((-1, 1))

initial_distribution = np.array((0.5, 0.5))

result = baum_welch(V, a, b, initial_distribution, n_iter=1)
print(result)   # I add the initial probability part, but I'm not sure the answer

{'a': array([[0.49354007, 0.50645993],
       [0.49321683, 0.50678317]]), 'b': array([[0.16707575, 0.27372847, 0.55919578],
       [0.24387812, 0.26637174, 0.48975014]]), 'initial_distribution': array([0.49213684, 0.50586316])}


### I want to revise the algorithm to use the $\epsilon$ to be the condition to stop, but it stop at a weird place.

In [114]:
def baum_welch2(V, a, b, initial_distribution, epi = 0.0001):
    M = a.shape[0]
    T = len(V)
    b_org = b.copy()
    
    while True:
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)
        
 
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator
        
        gamma = np.sum(xi, axis=1)
        a_new = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
        
        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)
 
        b_new = np.divide(b, denominator.reshape((-1, 1)))
        
        # check the difference is small enouth to break
        #a_diff = np.array(a) - np.array(a_new)     # try to use norm 2 to compare, but weird
        #b_diff = np.array(b_org) - np.array(b_new)
        #if np.linalg.norm(a_diff, 2) + np.linalg.norm(b_diff, 2) < epi:
        #    break
        #else:
        #    a = a_new
        #    b = b_new
        #    b_org = b.copy()
        alpha_2 = forward(V, a_new, b_new, initial_distribution)  # use the likelihood to compare
        if abs(alpha[-1].sum() - alpha_2[-1].sum()) < epi:
            print(abs(alpha[-1].sum() - alpha_2[-1].sum()))
            break
        else:
            a = a_new
            b = b_new
            b_org = b.copy()

    return {"a":a_new, "b":b_new}

In [128]:
a = np.ones((2, 2))
a = a / np.sum(a, axis=1)

b = np.array(((1, 3, 5), (2, 4, 6)))
b = b / np.sum(b, axis=1).reshape((-1, 1))

initial_distribution = np.array((0.5, 0.5))

result2 = baum_welch2(V, a, b, initial_distribution, 10**(-221))  # the criterior is weird, 
print(result2)

1.3044550522455806e-225
{'a': array([[0.49388933, 0.50611067],
       [0.49324199, 0.50675801]]), 'b': array([[0.16710908, 0.27371913, 0.55917179],
       [0.24387565, 0.26637796, 0.48974639]])}


Reference : 
*http://www.adeveloperdiary.com/data-science/machine-learning/derivation-and-implementation-of-baum-welch-algorithm-for-hidden-markov-model/* --> gives me the idea about how to implement it

*https://github.com/hamzarawal/HMM-Baum-Welch-Algorithm/blob/2c5883c42f171f3c54a9afa4ae85eeabb196dfde/baum-welch.py#L110* --> gives me the idea about using likelihood to compare ! but its data is small, so when data goes bigger, the criterior goes crazy ! 